In [9]:
import requests
import re
import numpy as np
from lxml import html
import time

In [10]:
def get_prx():
    prx = requests.get('http://spys.one/')
    tree = html.fromstring(prx.text)
    prx_lst = []
    for i in tree.xpath('.//body/table/tr/td/table/tr/td/table/tr/td/font/text()'):
        if re.match('^[\d, \.]+:\d+$', i):
            prx_lst.append(i)
    return prx_lst

In [11]:
search_str = ['игровая+майка', \
              'игровая+футболка',\
              'футболки+с+матча', \
              'футболки+после+матча', \
              'футболка+футболиста',\
              'футболка+игрока']

categories = ['', 'hobbi_i_otdyh/', 'odezhda_obuv_aksessuary/muzhskaya_odezhda/']
region = 'rossiya/'

vk_token = 'ec7bc07d2f5966fa9d3194cf8dbc0cd5f03645d06bf651061aa77936f402ce38d172bbca7bf3012697b1d'
avito_url = 'https://www.avito.ru/'

old_itm_key = dict()

In [12]:
prx_lst = get_prx()

In [13]:
print('Запуск..')

while True:
    for category in categories:
        for srch in search_str:
            avito = requests.get(avito_url+region+category, \
                                 params={'cd':1, 'q':srch, 's':104, 'user':1}, \
                                 timeout=10, \
                                 proxies={'http':np.random.choice(prx_lst)}
                                )
                
            print(avito.url)
            if avito.status_code != 200:
                continue
                    
            tree = html.fromstring(avito.text)
            snippets = tree.find_class('snippet-horizontal')
                
            if not snippets:
                continue
                    
            itm_key = snippets[0].get('data-item-id')
                
            if itm_key == None:
                continue
                    
            if (category, srch) in old_itm_key.keys():
                    
                if old_itm_key[(category, srch)] != itm_key:
                    print(old_itm_key[(category, srch)], itm_key, sep=':')
                    msg = avito_url+snippets[0].find_class('item-slider')[0].get('href')
                    vk = requests.post('https://api.vk.com/method/messages.send', \
                                       params={'access_token':vk_token, 'user_id':299193929, \
                                               'random_id':np.random.randint(0, 10000000000, dtype=np.int64),\
                                                'message':msg, 'v':'5.84'}, \
                                        timeout=10, \
                                        proxies={'http':np.random.choice(prx_lst)}
                                      )
                    print('message sended: ' + vk.text)
                        
            old_itm_key[(category, srch)]=itm_key
            time.sleep(10)

Запуск..
https://www.avito.ru/rossiya?cd=1&q=%D0%B8%D0%B3%D1%80%D0%BE%D0%B2%D0%B0%D1%8F%2B%D0%BC%D0%B0%D0%B9%D0%BA%D0%B0&s=104&user=1
https://www.avito.ru/rossiya?cd=1&q=%D0%B8%D0%B3%D1%80%D0%BE%D0%B2%D0%B0%D1%8F%2B%D1%84%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D0%BA%D0%B0&s=104&user=1
https://www.avito.ru/rossiya?cd=1&q=%D1%84%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D0%BA%D0%B8%2B%D1%81%2B%D0%BC%D0%B0%D1%82%D1%87%D0%B0&s=104&user=1
https://www.avito.ru/rossiya?cd=1&q=%D1%84%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D0%BA%D0%B8%2B%D0%BF%D0%BE%D1%81%D0%BB%D0%B5%2B%D0%BC%D0%B0%D1%82%D1%87%D0%B0&s=104&user=1
https://www.avito.ru/rossiya?cd=1&q=%D1%84%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D0%BA%D0%B0%2B%D1%84%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D0%B8%D1%81%D1%82%D0%B0&s=104&user=1
https://www.avito.ru/rossiya?cd=1&q=%D1%84%D1%83%D1%82%D0%B1%D0%BE%D0%BB%D0%BA%D0%B0%2B%D0%B8%D0%B3%D1%80%D0%BE%D0%BA%D0%B0&s=104&user=1
https://www.avito.ru/rossiya/hobbi_i_otdyh?cd=1&q=%D0%B8%D0%B3%D1%80%D0%BE%D0%B2%D0%B0%D1%8F%2B%D0%BC%D0%B0%D0%B9%D0%BA%D

ConnectionError: HTTPSConnectionPool(host='www.avito.ru', port=443): Max retries exceeded with url: /rossiya/hobbi_i_otdyh/?cd=1&q=%D0%B8%D0%B3%D1%80%D0%BE%D0%B2%D0%B0%D1%8F%2B%D0%BC%D0%B0%D0%B9%D0%BA%D0%B0&s=104&user=1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000215E67B8588>: Failed to establish a new connection: [WinError 10051] Сделана попытка выполнить операцию на сокете при отключенной сети'))